### 추천시스템 구현

1. Surprise -> 협업 필터링(Matrix Factorization 기반 추천)
2. Implicit -> Implicit Feedback 기반 추천 (ALS, BPR 모델)
3. LightFM -> Hybrid Recommendation (협업 + 콘텐츠 기반 추천)

### STEP 1 : Surprise 활용한 협업 필터링 추천

Surprise 라이브러리는 사용자-아이템 평점 데이터 기반 추천 모델을 쉽게 구현할 수 있도록 돕는 라이브러리입니다.\
여기서는 SVD(Singular Value Decomposition)를 활용하여 추천을 수행하겠습니다.

- 구현 내용
1. MovieLens 데이터셋을 사용하여 SVD 기반 추천 모델 학습
2. 특정 사용자에게 맞춤형 영화 추천

#### 1. Surprise 설치 및 데이터 로드

먼저 Surprise 라이브러리를 설치하고 MovieLens 데이터를 불러옵니다.\
이후 SVD를 활용해 모델을 학습하고, 사용자에게 영화를 추천하는 코드입니다.

In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
import surprise
from surprise import SVD, Dataset
from surprise.model_selection import cross_validate

# MovieLens 데이터 로드
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\Seonghyeon/.surprise_data/ml-100k


In [6]:
# SVD 모델 학습
model = SVD()
cross_validate(model, data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9368  0.9326  0.9401  0.9310  0.9352  0.9351  0.0032  
MAE (testset)     0.7347  0.7351  0.7402  0.7350  0.7411  0.7372  0.0028  
Fit time          0.60    0.62    0.72    0.59    0.60    0.63    0.05    
Test time         0.08    0.13    0.06    0.13    0.06    0.09    0.03    


{'test_rmse': array([0.93682966, 0.93258815, 0.94012084, 0.93102914, 0.93515786]),
 'test_mae': array([0.73468098, 0.73507174, 0.74020395, 0.73495238, 0.74105339]),
 'fit_time': (0.6001081466674805,
  0.6166708469390869,
  0.723581075668335,
  0.5862545967102051,
  0.6026055812835693),
 'test_time': (0.07545828819274902,
  0.12703752517700195,
  0.0584561824798584,
  0.1253972053527832,
  0.06400203704833984)}

In [7]:
# 특정 사용자를 위한 영화 추천
# 학습 데이터 생성
trainset = data.build_full_trainset()
model.fit(trainset)

# 특정 사용자(user = 196)에게 영화 추천
user_id = str(196)
items = trainset.all_items()
predictions = [model.predict(user_id, trainset.to_raw_iid(i)) for i in items]

# 예측 평점이 높은 순으로 정렬
top_n = sorted(predictions, key=lambda x:x.est, reverse=True)[:10]

# 추천 결과 출력
print('추천영화 목록 :')
for pred in top_n:
    print(f"영화 : {pred.iid}, 예상평점 : {pred.est:.2f}")

추천영화 목록 :
영화 : 318, 예상평점 : 4.61
영화 : 64, 예상평점 : 4.57
영화 : 408, 예상평점 : 4.55
영화 : 480, 예상평점 : 4.55
영화 : 98, 예상평점 : 4.47
영화 : 647, 예상평점 : 4.46
영화 : 357, 예상평점 : 4.46
영화 : 127, 예상평점 : 4.46
영화 : 313, 예상평점 : 4.45
영화 : 169, 예상평점 : 4.44


In [8]:
# 하이퍼 파라미터 튜닝도 가능
model = SVD(n_factors = 100, n_epochs=20, lr_all = 0.005)

### STEP 2 : Implicit 활용한 Implicit Feedback 기반 추천

Implicit 라이브러리는 사용자의 명시적 피드백(평점) 대신 클릭, 조회수, 구매 여부 같은 암묵적 피드백(Implicit Feedback) 을 기반으로 추천하는 모델을 제공합니다.

여기서는 ALS (Alternating Least Squares) 모델을 사용하여 추천을 수행해보겠습니다.

In [1]:
# 필요한 라이브러리 불러오기
import implicit
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

c:\Users\Seonghyeon\anaconda3\envs\main\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 샘플 데이터 생성 (User-Item Interaction Matrix)
user_item_matrix = np.array([
    [5, 0, 3, 0, 2],
    [4, 0, 0, 1, 3],
    [1, 1, 0, 5, 0],
    [0, 0, 4, 4, 0]
])

# 희소 행렬로 변환
user_item_sparse = csr_matrix(user_item_matrix)

# ALS 모델 학습
model = implicit.als.AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20)
model.fit(user_item_sparse)

# 특정 사용자에게 추천
user_id = 1
recommendations = model.recommend(user_id, user_item_sparse[user_id], N=5)

# 추천 결과 출력
print("추천 아이템 목록:", recommendations)

c:\Users\Seonghyeon\anaconda3\envs\main\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
100%|██████████| 20/20 [00:00<00:00, 2892.72it/s]

추천 아이템 목록: (array([2, 1, 4, 3, 0]), array([ 3.1607822e-02,  2.7578674e-02, -3.4028235e+38, -3.4028235e+38,
       -3.4028235e+38], dtype=float32))


### Step 3: LightFM을 활용한 Hybrid Recommendation

LightFM은 협업 필터링과 콘텐츠 기반 추천을 결합한 하이브리드 추천 모델을 제공합니다.
이를 통해 사용자 및 아이템 메타데이터를 활용하여 보다 정교한 추천이 가능합니다.

In [ ]:
# 필요한 라이브러리 불러오기
import numpy as np
from lightfm import LightFM
from lightfm.datasets import fetch_movielens

In [ ]:
# MovieLens 데이터 로드
data = fetch_movielens(min_rating=4.0)

# LightFM 모델 생성 (Hybrid Model: 협업 필터링 + 콘텐츠 기반 추천)
model = LightFM(loss='warp')  # WARP(Weighted Approximate-Rank Pairwise) 랭킹 알고리즘 적용
model.fit(data['train'], epochs=10, num_threads=2)

# 특정 사용자에게 영화 추천
def recommend(model, data, user_id, n=5):
    scores = model.predict(user_id, np.arange(data['train'].shape[1]))
    top_items = np.argsort(-scores)[:n]
    return top_items

# 사용자 ID 3에게 추천
user_id = 3
recommended_items = recommend(model, data, user_id)
print("추천 영화 ID:", recommended_items)